# Starbucks Capstone Challenge

# Introduction


For this project, we will have an opportunity to analyse Starbucks mobile app customer base data and their responses towards promotion offers. First we will explore, clean the data and then make exploratory analysis to find out customer based statistic. Subseqently, we aim to make on what the app should make recommendation offer to the new customer on the app with a machine learning model

## About Starbucks and Mobile App Program

### Starbuck
Starbucks Corporation is an American multinational chain of coffeehouses and roastery reserves headquartered in Seattle, Washington since 1971[1]. It is currently the world's largest coffeehouse chain store, serving a variety of beverages from hot/cold coffee/tea to the In addition to drinks and food, many stores carry Starbucks' official merchandise, such as mugs, tumblers, scoops, and coffee presses. Not limited to store front sales, drinks (canned drink, expresso capsules) sporting Starbuck label has appeared in supermarket and online shops[2]

### Mobile Application
Starbucks is one of the early adopter of mobile application. In May 2008, a loyalty program was introduced for Starbucks Card registered users offering perks such as free Wi-Fi Internet access etc. 3 Years on, it began beta testing its mobile app for the Starbucks card, offering prepaid services. Subsequently, Starbucks released its complete mobile platform by January 2011. By December 2011, the number of mobile transactions exceeded 26 million. It is estimated that 30% of the Starbuck transactions is through mobile compared to Apple Pay, which is only used by ~5% of customers in 2018 [3]

## Information on Data Set
The provided data set for this project contains simulated data (containing transactions, customer spread and offer types) which mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks and not every user will have the same offer offered to the next

#### Validity Period
Every offer has a validity period before the offer expires. For example, a BOGO offer might be valid for only 5 days. The data set contains also informational offers which are merely providing information about a product. Accordingly, if an informational offer has 7 days of validity, one can assumed the customer is under the influence of the offer for 7 days after receiving the advertisement. It can be shown from the transactional data shows the timestamp of each purchase as well as the amount of money spent when user make purchases on the app. The transactional data also has a record for each offer that a user receives, a record for when a user actually views the offer and finally when a user completes an offer. 

Conversely, user should also note that it is possible that a customer using the app might make a purchase through the app without having received an offer or seen an offer.

#### Rewards System
To give an example how the reward process works, a user could receive a discount offer buy 10 dollars get 2 off. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer. 

It has also to be considered that a customer can receive an offer, never actually views the offer, and still completes the offer. For example, a customer might receive the "buy 10 dollars get 2 dollars off offer", but never opens the offer during the 10 day validity period, while he spends 15 dollars during those ten days. Thus there will be an offer completion record in the data set while the customer was not influenced by the offer.[2]

## On Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

# Part I : Define the business problem

The first part of our studies is to define our business problem
<ol>
    <li> What is the age, gender and income demographics of the customer group? Who formed the majority of the Starbucks customer base ?</li>
    <li> Which is the most effective channel in dispensing out information </li>
    <li> Which group (age, income, membership length) of customers appear to be more responsive towards the promotion
    <li> Is there link between the offer given and the target group? </li>
    <li> Which type of promotional offer most appeal to which group of customer?</li>
    <li>Can we build a recommendation engine to recommend promotional offer with good uptake rate to new customers based on their demographical data age, income, registration date and gender?</li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt

%matplotlib inline

# read in the json files
portfolio   = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile     = pd.read_json('data/profile.json',   orient='records', lines=True)
transcript  = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
## update the pandas package as suggested above

# if (float(pd.__version__) > 1.53):
#     print('version is greater')
# !conda update pandas

# Part II : Analyze


We will analyze the problem(with the 3 pandas DataFrame) through visualizations and data exploration to have a better understanding of what algorithms and features are appropriate for solving it.

### Portfolio

In [3]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [4]:
portfolio.describe(include='all')

,reward,channels,difficulty,duration,offer_type,id
count,10.000000,10,10.000000,10.000000,10,10
unique,NaN,4,NaN,NaN,3,10
top,NaN,"[web, email, mobile, social]",NaN,NaN,bogo,ae264e3637204a6fb9bb56bc8210ddfd
freq,NaN,4,NaN,NaN,4,1
mean,4.200000,NaN,7.700000,6.500000,NaN,NaN
std,3.583915,NaN,5.831905,2.321398,NaN,NaN
min,0.000000,NaN,0.000000,3.000000,NaN,NaN
25%,2.000000,NaN,5.000000,5.000000,NaN,NaN
50%,4.000000,NaN,8.500000,7.000000,NaN,NaN
75%,5.000000,NaN,10.000000,7.000000,NaN,NaN


In [5]:
portfolio.isna().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

### Description of Portfolio Data

This data contains 10 campaign offers that is run together with Starbuck Customer, of which there are 3 types of promotion types being offered and the campaign offers is run with up to 4 types of channels. There is no missing data in the set and so we did not need to impute any value. Below tells of the detail for each column

**portfolio.json schema**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer, - cost to consumer
* reward (int) - reward given for completing an offer - cost to Starbuck
* duration (int) - time for offer to be open, in days
* channels (list of strings), ie web, email, mobile, social

### The following cleansing work would be neccessary
<ol> 
    <li> To expand the channels column into multiple columns based on the list atttribute: namely web, email, mobile and social  
    <li> To drop some of the channels namely channels and email. Email is used in every info campaign while channels have been expanded and is redundant
    <li> To expand the offer_type, a form of categorical data, into individual columns
    <li> Convert the duration to number of hours instead of days
    <li> Normalize some of the value 
</ol>

### Profile

In [6]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [7]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


To find out how many person is of age 118

In [8]:
profile.age[profile.age==118].count()

2175

In [9]:
profile.isna().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

The number of people with age 118 matches with the number of people without gender and without income data

### Description of Profile Data

A glimpse at the data show there is a number of customers, (2175 to be exact) at 118 year old! A small trivial[4], the oldest known living person  has managed to live a year of 122 and the number of people who have live to that age is not more than 10. Starbucks could not possibly get so many customer who is of 118. <br>

A plausible explanation could be that the customer refuse to leave his/her identity information. The number of customer who did not have their gender, income or age is the same, 2175. We might be wanting to remove this set of user, even though it represented about 10% of the total user database

We might also want to convert the became member 

**profile.json schema**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

The following cleansing work would be neccessary

<ol> 
    <li> To drop the members with 118 years of age     
    <li> To convert "became_member_on" to "membership". Would need to change from string format YYYYMMDD representatation to one that is based on year they have been member
    <li> Normalize some of the value 
</ol>

### Transcript

In [10]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [11]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [12]:
transcript.isna().sum()

person    0
event     0
value     0
time      0
dtype: int64

In [13]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

### Description of Transcript Data



transcript.json

    event (str) - record description of type transaction, offer received, offer viewed, offer completed
    person (str) - customer id
    time (int) - time in hours since start of test. The data begins at time t=0
    value - (dict of strings) - either an offer id or transaction amount depending on the record


The following cleansing work would be neccessary 

<ol>
    <li> To remove transaction of those users which have been removed in profile json 
    <li> To expand the event column, a form of categorical data, into individual binary columns of transaction, offer received, offer viewed, offer completed
    <li> To make matter simple, formed a new column call campaign_success based on customer would have to view the offer and complete the offer, then it would consider as campaign success while completion of offer without actually viewing the offer would consider to be a failure
    <li> To extract out the value type into either one indicating of offer_id or transaction amount. Consideration is also taken to drop those transaction amount data
    <li> Normalize some of the value 
</ol>

Finally then combine the 3 dataframe to form a master dataframe

# Part III :  Data Cleansing

We perform cleaning and engineering for all data sets (portfolio, profile, transcript). In summary, we will focus on the below work direction:

<ul>
    <li> convert dtype of features where necessary eg from datetime to string or vice versa </li>
    <li> drop rows with missing data or outliers </li>
    <li> drop rows with duplicated data </li>
    <li> group data for further data analysis </li>
    <li> create new features with binaries from categorical variables </li>
    <li> normalize data for columns with a different range of values </li>
</ul>

Cleaning up the portfolio dataset

In [14]:
## copy a copy of the portfolio dataset
portfolio_df  = portfolio.copy()

Create new columns(web, emaiil,mobile & social) from channels

In [15]:
channels_list  =  ['web', 'email', 'mobile', 'social'] 

for index in channels_list: 
    portfolio_df[index] = portfolio_df.channels.apply(lambda x: (index in x)+0)

print(portfolio_df)

   reward                      channels  difficulty  duration     offer_type  \
0      10       [email, mobile, social]          10         7           bogo   
1      10  [web, email, mobile, social]          10         5           bogo   
2       0          [web, email, mobile]           0         4  informational   
3       5          [web, email, mobile]           5         7           bogo   
4       5                  [web, email]          20        10       discount   
5       3  [web, email, mobile, social]           7         7       discount   
6       2  [web, email, mobile, social]          10        10       discount   
7       0       [email, mobile, social]           0         3  informational   
8       5  [web, email, mobile, social]           5         5           bogo   
9       2          [web, email, mobile]          10         7       discount   

                                 id  web  email  mobile  social  
0  ae264e3637204a6fb9bb56bc8210ddfd    0      1      

Convert offer_type from categorical into numerical data

In [16]:
portfolio_df.offer_type = portfolio_df.offer_type.replace([ 'informational', 'bogo', 'discount'],
                          [0, 1, 2])

Changing the duration from day into hours

In [17]:
portfolio_df.duration  = portfolio_df.duration * 24

Adding three more columns -- Offered, Viewed, Complete

In [18]:
portfolio_df['Viewed'] = 0
portfolio_df['Completed'] = 0
portfolio_df['Offered'] = 0

#### Cleaning up profile dataframe

changing the "became_member_on" to member since the earliest date

In [19]:
## make a copy of profile
profile_df = profile.copy()

Removing the anonmynous user with age 118, no gender and income information

In [20]:
profile_df = profile_df[profile_df.age!=118]
profile_df_size = 17000-2175

### sanity check on profile - after removing
assert profile_df.shape[0] >= profile_df_size , "the size is not right"

Adding columns for future usage

In [21]:
profile_df['Completed'] = 0
profile_df['Viewed'] = 0
profile_df['Offered'] = 0

Changing the data in column "became_member_on" to "membership"

In [22]:
## rename the columns
profile_df.rename(columns = {'became_member_on':'membership'}, inplace=True)
                             
base_year  = int(profile_df.membership.max() /10000) 
profile_df['membership']= profile_df.membership.apply(lambda x: base_year - int(x/10000))

print(profile_df)

      gender  age                                id  membership    income  \
1          F   55  0610b486422d4921ae7d2bf64640c50b           1  112000.0   
3          F   75  78afa995795e4d85b5d9ceeca43f5fef           1  100000.0   
5          M   68  e2127556f4f64592b11af22de27a7932           0   70000.0   
8          M   65  389bc3fa690240e798340f5a15918d5c           0   53000.0   
12         M   58  2eeac8d8feae4a8cad5a6af0499a211d           1   51000.0   
...      ...  ...                               ...         ...       ...   
16995      F   45  6d5f3a774f3d4714ab0c092238f3a1d7           0   54000.0   
16996      M   61  2cb4f97358b841b9a9773a7aa05a9d77           0   72000.0   
16997      M   49  01d26f638c274aa0b965d24cefe3183f           1   73000.0   
16998      F   83  9dc1421481194dcd9400aec7c9ae6366           2   50000.0   
16999      F   62  e4052622e5ba45a8b96b59aba68cf068           1   82000.0   

       Completed  Viewed  Offered  
1              0       0        0  
3  

Creating a master dataframe to hold all the data

#### Cleaning up transcript dataframe

In [23]:
transcript_df = transcript.copy()

Removing those records whose user does not appear in the new profile dataframe

In [24]:
transcript_df = transcript_df.value.apply(lambda x: (index in x)+0)

transcript_df = transcript[transcript['person'].isin(profile_df['id'])]

transcript_df.count()

person    272762
event     272762
value     272762
time      272762
dtype: int64

We have almost removed 10% of the records in transcript

In [25]:
transcript_df.event.value_counts()

transaction        123957
offer received      66501
offer viewed        49860
offer completed     32444
Name: event, dtype: int64

Changing the event categories into individual columns (binary) and then rename them

In [26]:
event_list  =  ['transaction', 'offer received', 'offer viewed', 'offer completed'] 

transcript_df_events = pd.DataFrame()

for index in event_list: 
    transcript_df_events[index] = transcript_df.event.apply(lambda x: 1 if x == index else 0 )

### rename 
transcript_df_events.rename(columns={"offer received": "offer_received",
                                     "offer viewed": "offer_viewed", 
                                     "offer completed": "offer_completed"}, inplace=True)

### print out for view
print(transcript_df_events)

        transaction  offer_received  offer_viewed  offer_completed
0                 0               1             0                0
2                 0               1             0                0
5                 0               1             0                0
7                 0               1             0                0
8                 0               1             0                0
...             ...             ...           ...              ...
306527            0               0             0                1
306529            1               0             0                0
306530            1               0             0                0
306531            1               0             0                0
306532            1               0             0                0

[272762 rows x 4 columns]


In [27]:
### find out the events categories and number
transcript_df_events.sum()

transaction        123957
offer_received      66501
offer_viewed        49860
offer_completed     32444
dtype: int64

Changing the values categories into individual columns (binary)

In [28]:
value_list  =  ['amount', 'offer_id', 'offer id'] 

transcript_df_value = pd.DataFrame()

for index in value_list: 
    transcript_df_value[index] = transcript_df.value.apply(lambda x: x.get(index) if x.get(index) != None else "" )

transcript_df_value['offer_id']   = transcript_df_value['offer_id']  + transcript_df_value['offer id'] 
transcript_df_value.drop('offer id', axis=1, inplace=True)
print(transcript_df_value)

       amount                          offer_id
0              9b98b8c7a33c4b65b9aebfe6a799e6d9
2              2906b810c7d4411798c6938adc9daaa5
5              f19421c1d4aa40978ebb69ca19b0e20d
7              3f207df678b143eea3cee63160fa8bed
8              0b1e1539f2cc45b7b9fa7c272da2e1d7
...       ...                               ...
306527         fafdcd668e3743c1bb461111dcafc2a4
306529   1.59                                  
306530   9.53                                  
306531   3.61                                  
306532   3.53                                  

[272762 rows x 2 columns]


Concatenating the dataframe

In [29]:
frame = [transcript_df, transcript_df_events,transcript_df_value]

transcript_df = pd.concat(frame, axis=1)

In [30]:
transcript_df.isna().sum()

person             0
event              0
value              0
time               0
transaction        0
offer_received     0
offer_viewed       0
offer_completed    0
amount             0
offer_id           0
dtype: int64

In [31]:
print(transcript_df)

                                  person            event  \
0       78afa995795e4d85b5d9ceeca43f5fef   offer received   
2       e2127556f4f64592b11af22de27a7932   offer received   
5       389bc3fa690240e798340f5a15918d5c   offer received   
7       2eeac8d8feae4a8cad5a6af0499a211d   offer received   
8       aa4862eba776480b8bb9c68455b8c2e1   offer received   
...                                  ...              ...   
306527  24f56b5e1849462093931b164eb803b5  offer completed   
306529  b3a1272bc9904337b331bf348c3e8c17      transaction   
306530  68213b08d99a4ae1b0dcb72aebd9aa35      transaction   
306531  a00058cf10334a308c68e7631c529907      transaction   
306532  76ddbd6576844afe811f1a3c0fbb5bec      transaction   

                                                    value  time  transaction  \
0        {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0            0   
2        {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0            0   
5        {'offer id': 'f194

We would also need to filter out customers who completed the offer without recieving or viewing the offer to give a more balanced view. However first we would need to create a new dataframe

In [32]:
transcript_df.offer_received.sum()

66501

Finally we would want to create another DataFrame called offer which is based on every single offer made in the 10 campaign. The dataframe would include information from the 3 dataframe: 
<ul>
    <li> portfolio
    <li> profile
    <li> transaction
</ul>

From transaction data frame, we would pull the following rows that have recieved the offer and the following columns
<ul>
    <li>person
    <li>offer_id
    <li>offer_viewed 
    <li>offer_completed
    <li>offer_type
    <li>amount
</ul>

From portfolio data frame, we would pull the following columns
<ul>
     <li>reward
     <li>difficulty 
     <li>duration  
     <li>web
     <li>mobile
     <li>social
</ul>

From profile data frame, we would pull the following columns
<ul>
     <li>gender
     <li>age 
     <li>income  
     <li>membership
</ul>

In [33]:
def findViewed(df, offer_id):
    result = False
    if (df.loc[df.offer_id == offer_id].offer_viewed ==1).any():
        result = True
    
    return result

In [34]:
### we made a copy of whatever we did until now
offer = transcript_df.copy()

#### get all the offer that were send out
offer = offer[offer.offer_received==1]

offer = offer.drop(['offer_received','value', 'time', 'transaction', 'amount'], axis=1)

print(offer.person.count())

66501


In [37]:
transcript_df_person_offerid = transcript_df.groupby(['person', 'offer_id'])

for key, item in transcript_df_person_offerid:
    tmp = transcript_df_person_offerid.get_group(key)
    if tmp.offer_completed.sum() == 1:
        if tmp.offer_viewed.sum() == 1:
            offer.loc[offer.offer_id==key[1], 'offer_completed'] = 1
            offer.loc[offer.offer_id==key[1], 'offer_viewed'] = 1
        else:
            offer.loc[offer.offer_id==key[1], 'offer_completed'] = 0
            offer.loc[offer.offer_id==key[1], 'offer_viewed'] = 0
    if tmp.offer_viewed.sum() == 1:
        offer.loc[offer.offer_id==key[1], 'offer_viewed'] = 1
    






66501

In [44]:
portfolio_df = portfolio_df.rename(columns={'id': 'offer_id'})

portfolio_df

,reward,channels,difficulty,duration,offer_type,offer_id,web,email,mobile,social,Viewed,Completed,Offered
0,10,"[email, mobile, social]",10,168,1,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,0,0,0
1,10,"[web, email, mobile, social]",10,120,1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,0,0,0
2,0,"[web, email, mobile]",0,96,0,3f207df678b143eea3cee63160fa8bed,1,1,1,0,0,0,0
3,5,"[web, email, mobile]",5,168,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,0,0,0
4,5,"[web, email]",20,240,2,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,0,0,0
5,3,"[web, email, mobile, social]",7,168,2,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,0,0
6,2,"[web, email, mobile, social]",10,240,2,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,0,0
7,0,"[email, mobile, social]",0,72,0,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1,0,0,0
8,5,"[web, email, mobile, social]",5,120,1,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,0,0,0
9,2,"[web, email, mobile]",10,168,2,2906b810c7d4411798c6938adc9daaa5,1,1,1,0,0,0,0


In [45]:
offer


,person,event,offer_viewed,offer_completed,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
2,e2127556f4f64592b11af22de27a7932,offer received,0,0,2906b810c7d4411798c6938adc9daaa5
5,389bc3fa690240e798340f5a15918d5c,offer received,1,1,f19421c1d4aa40978ebb69ca19b0e20d
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,1,0,3f207df678b143eea3cee63160fa8bed
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,1,1,0b1e1539f2cc45b7b9fa7c272da2e1d7
...,...,...,...,...,...
257882,d087c473b4d247ccb0abfef59ba12b0e,offer received,1,1,ae264e3637204a6fb9bb56bc8210ddfd
257883,cb23b66c56f64b109d673d5e56574529,offer received,0,0,2906b810c7d4411798c6938adc9daaa5
257884,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,1,1,2298d6c36e964ae4a3e7e9706d1fb8c2
257885,9dc1421481194dcd9400aec7c9ae6366,offer received,1,1,ae264e3637204a6fb9bb56bc8210ddfd


In [46]:
pd.merge(offer, portfolio_df, how='left')

,person,event,offer_viewed,offer_completed,offer_id,reward,channels,difficulty,duration,offer_type,web,email,mobile,social,Viewed,Completed,Offered
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,"[web, email, mobile]",5,168,1,1,1,1,0,0,0,0
1,e2127556f4f64592b11af22de27a7932,offer received,0,0,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,2,1,1,1,0,0,0,0
2,389bc3fa690240e798340f5a15918d5c,offer received,1,1,f19421c1d4aa40978ebb69ca19b0e20d,5,"[web, email, mobile, social]",5,120,1,1,1,1,1,0,0,0
3,2eeac8d8feae4a8cad5a6af0499a211d,offer received,1,0,3f207df678b143eea3cee63160fa8bed,0,"[web, email, mobile]",0,96,0,1,1,1,0,0,0,0
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,1,1,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,"[web, email]",20,240,2,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66496,d087c473b4d247ccb0abfef59ba12b0e,offer received,1,1,ae264e3637204a6fb9bb56bc8210ddfd,10,"[email, mobile, social]",10,168,1,0,1,1,1,0,0,0
66497,cb23b66c56f64b109d673d5e56574529,offer received,0,0,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,2,1,1,1,0,0,0,0
66498,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,1,1,2298d6c36e964ae4a3e7e9706d1fb8c2,3,"[web, email, mobile, social]",7,168,2,1,1,1,1,0,0,0
66499,9dc1421481194dcd9400aec7c9ae6366,offer received,1,1,ae264e3637204a6fb9bb56bc8210ddfd,10,"[email, mobile, social]",10,168,1,0,1,1,1,0,0,0


In [ ]:
def findViewed(df, offer_id):
    result = False
    if (df.loc[df.offer_id == offer_id].offer_viewed ==1).any():
        result = True
    
    return result

transcript_df_person = transcript_df.groupby('person')


### lots of heavy processing down here
for key, item in transcript_df_person:
    print(f'The customer id is {key}')
    tmp = transcript_df_person.get_group(key)
    count = 0
    vcount = 0
    for index, row in tmp.iterrows():
        if row.offer_completed == 1:
            search_string = row.offer_id
            if findViewed(tmp, search_string) == True:
                print(f'Offer {search_string} is completed for customer {key}')
                count = count + 1
                portfolio_df.loc[portfolio_df.id == search_string,'Complete'] = portfolio_df.loc[portfolio_df.id == search_string,'Complete'] + 1
            else:
                ### correction
                transcript_df.loc[index].offer_completed = 0
        if row.offer_viewed == 1:
            search_string = row.offer_id
            # the key is the advert 
            portfolio_df.loc[portfolio_df.id == search_string,'Viewed'] = portfolio_df.loc[portfolio_df.id == key,'Viewed'] + 1
    
    
    profile_df.loc[ profile_df.id == key,'Completion'] = count
    profile_df.loc[ profile_df.id == key,'Viewed'] = tmp.offer_viewed.sum()
    profile_df.loc[ profile_df.id == key,'Offered'] = tmp.offer_received.sum()
    
    

In [ ]:
transcript_df_offer = transcript_df[transcript_df['transaction']==0]

transcript_df_id = transcript_df_offer.groupby('offer_id')

for key, item in transcript_df_id:
    tmp = transcript_df_id.get_group(key)
#     print(f'offer id {key} has the  {tmp.offer_received.sum()} offered')
    portfolio_df.loc[portfolio_df.id== key, "Offered"] = tmp.offer_received.sum()
    portfolio_df.loc[portfolio_df.id== key, "Viewed"]  = tmp.offer_viewed.sum()  
    

In [ ]:
print(portfolio_df)

In [ ]:
print(profile_df)

Finally we would want to create another DataFrame called offer which is based on every single offer made in the 10 campaign. The dataframe would include information from the 3 dataframe: 
<ul>
    <li> portfolio
    <li> profile
    <li> transaction
</ul>

From transaction data frame, we would pull the following rows that have recieved the offer and the following columns
<ul>
    <li>person
    <li>offer_id
    <li>offer_viewed 
    <li>offer_completed
    <li>offer_type
    <li>amount
</ul>

From portfolio data frame, we would pull the following columns
<ul>
     <li>reward
     <li>difficulty 
     <li>duration  
     <li>web
     <li>mobile
     <li>social
</ul>

From profile data frame, we would pull the following columns
<ul>
     <li>gender
     <li>age 
     <li>income  
     <li>membership
</ul>


## Exploratory Data Analysis

##### What is the age, gender and income demographics of the customer group? 
<ul>
    <li>Who formed the majority of the Starbucks customer base ?
    <li>How should we divide up the customer into grouping for later analysis?

In [ ]:
plt.subplots(figsize=(20, 14));

### Length of Membership
plt.subplot(2,2,1)
plt.title("Distribution of Membership Duration\n",
         fontweight ="bold")
num_bins = 6
n, bins, patches = plt.hist(profile_df['membership'], num_bins, 
                            color ='tab:blue',
                            alpha = 0.8)

plt.xlabel('Years of Being Member', fontweight ="bold")
plt.ylabel('Customer', fontweight ="bold")


### Age Distribution
plt.subplot(2,2,2)
plt.title('Histogram Plot of Customer Age Range\n',
          fontweight ="bold")
num_bins = 12
n, bins, patches = plt.hist(profile_df['age'], num_bins, 
                            color ='tab:blue',
                            alpha = 0.8)

plt.xlabel('Age', fontweight ="bold")
plt.ylabel('Customers', fontweight ="bold")
  
### Income Distribution
plt.subplot(2,2,3)
plt.title("Income Distribution\n", fontweight ="bold")

num_bins = 6
n, bins, patches= plt.hist(profile_df['income'], num_bins, 
                            color ='tab:blue',
                            alpha = 0.8)
plt.xlabel('Income', fontweight ="bold")
plt.ylabel('Customers', fontweight ="bold")

### Gender Distribution
plt.subplot(2,2,4)
gender_m = profile_df[profile_df['gender'] == 'M'].gender.count()
gender_f = profile_df[profile_df['gender'] == 'F'].gender.count()
gender = [gender_m , gender_f ]
labels=['Male', 'Female']

plt.title("Gender Distribution", fontweight ="bold")
plt.pie(gender, labels=labels,  labeldistance=0.6, textprops={'fontsize': 18})

plt.show()

In terms of gender, there is more men in the customer group than women. At the same time, it is interesting that most of the customer in the group seem to be around 40-70, peaking round 50+, a particularly mature customer group. It might be because the campaign intentionally target this group of mature customer, being naturally high power spending group(due to age). The mode of income is around 60000+

Dividing the profile dataframe into 3 different income group, namely:
<ol> 
    <li> from 53000 and below 
    <li> above 53000 and below 74000
    <li> above 74000
</ol>
    
Let us see the distribution of gender and age (which we shall use the quartile percentile age  as describe in the dataframe) within this group


In [ ]:
plt.subplots(figsize=(20, 8))

income_1  = profile_df.loc[profile.income <= 53000]
income_2  = profile_df.loc[(profile.income > 53000) & (profile.income <= 73000)]
income_3  = profile_df.loc[profile.income > 73000]

### Gender within the group
count_grp1   =  income_1.gender.count()
count_grp2   =  income_2.gender.count()
count_grp3   =  income_3.gender.count()

m_count_grp1 =  income_1.loc[income_1.gender== 'M'].gender.count()
f_count_grp1 =  income_1.loc[income_1.gender== 'F'].gender.count()
m_count_grp2 =  income_2.loc[income_2.gender== 'M'].gender.count()
f_count_grp2 =  income_2.loc[income_2.gender== 'F'].gender.count()
m_count_grp3 =  income_3.loc[income_3.gender== 'M'].gender.count()
f_count_grp3 =  income_3.loc[income_3.gender== 'F'].gender.count()

income =[m_count_grp1, f_count_grp1, count_grp1,
         m_count_grp2, f_count_grp2, count_grp2,
         m_count_grp3, f_count_grp3, count_grp3]
index  =['m_count_grp1', 'f_count_grp1', 'count_grp1',
         'm_count_grp2', 'f_count_grp2', 'count_grp2',
         'm_count_grp3', 'f_count_grp3', 'count_grp3']

plt.subplot(1,2,1)

color  = ['tab:blue', 'tab:pink', 'tab:green',
          'tab:blue', 'tab:pink', 'tab:green',
          'tab:blue', 'tab:pink', 'tab:green']

plt.title("Distribution of Gender Among The Income Group\n",
         fontweight ="bold")

plt.ylabel("No of People")
plt.xlabel("Income group")

plt.bar(index, height=income, color = color)

plt.xticks(rotation=45)

### Age within the group
count_1Q_grp1   =  income_1.loc[income_1.age <= 45].age.count()
count_2Q_grp1   =  income_1.loc[(income_1.age > 45) & (income_1.age <= 58)].age.count()
count_3Q_grp1   =  income_1.loc[(income_1.age > 58) & (income_1.age <= 73)].age.count()
count_4Q_grp1   =  income_1.loc[income_1.age >73].age.count()
count_1Q_grp2   =  income_2.loc[income_2.age <= 45].age.count()
count_2Q_grp2   =  income_2.loc[(income_2.age > 45) & (income_2.age <= 58)].age.count()
count_3Q_grp2   =  income_2.loc[(income_2.age > 58) & (income_2.age <= 73)].age.count()
count_4Q_grp2   =  income_2.loc[income_2.age >73].age.count()
count_1Q_grp3   =  income_3.loc[income_3.age <= 45].age.count()
count_2Q_grp3   =  income_3.loc[(income_3.age > 45) & (income_3.age <= 58)].age.count()
count_3Q_grp3   =  income_3.loc[(income_3.age > 58) & (income_3.age <= 73)].age.count()
count_4Q_grp3   =  income_3.loc[income_3.age >73].age.count()


plt.subplot(1,2,2)

color  = ['tab:orange', 'tab:purple', 'tab:olive', "tab:cyan",
          'tab:orange', 'tab:purple', 'tab:olive', "tab:cyan",
          'tab:orange', 'tab:purple', 'tab:olive', "tab:cyan"]
income =[count_1Q_grp1, count_2Q_grp1, count_3Q_grp1, count_4Q_grp1, 
         count_1Q_grp2, count_2Q_grp2, count_3Q_grp2, count_4Q_grp2,
         count_1Q_grp3, count_2Q_grp3, count_3Q_grp3, count_4Q_grp3]

index  =['count_1Q_grp1', 'count_2Q_grp1','count_3Q_grp1', 'count_4Q_grp1',
         'count_1Q_grp2', 'count_2Q_grp2', 'count_3Q_grp2', 'count_4Q_grp2',
         'count_1Q_grp3', 'count_2Q_grp3', 'count_3Q_grp3', 'count_4Q_grp3',]

plt.bar(index, height=income, color = color)
plt.xticks(rotation=45)

plt.title("Distribution of Customer Age Among The Income Group\n",
         fontweight ="bold")

plt.ylabel("No of People")
plt.xlabel("Customer Age group")
plt.show()

Let us first talk about the age grouping within the income group. 

Observation
<ul>
    <li>As expected, the youngest of the group dominates the lower income (as probably they have not been in workforce for too long) and they almost twice as many when compared to the rest.
    <li>The rest of the age-group exhibit increasing number of people when the income goes higher. 
    <li>There is almost no difference between the number of people in the 3 income group of age group 45-58 and 58-73.        
</ul>

For the income plot,  there is more males than female are found in the lower and mid-income level but the trend reverse for the high-level income group. This may be due to that at higher income group, it will require typically less manual and technical work which male could excel in but rather in sales and management jobs which females excel in


In [ ]:
def is_equal_variance(d1, d2, alpha):
    """
    Method for performing stats.levene test on two input samples. 
    As returns we get a boolian if sample variances are equal.

    Args:
        d1, d2 (pd.DataSeries): input sample data
        alpha (float): alpha value as proportion 

    Returns:
        boolian: true if variances between input distributions are equal (p > alpha), false otherwise (p <= alpha)
        W: test statistic as float
        p: p value for the test
    """
    ### check if variances are equal   
    W, p = stats.levene(d1, d2)
    if p <= alpha:
        print(f"group variances unequal: W = {W:.4f}, p = {p:.4f}")
        return False        
    else:
        print(f"group variances equal: W = {W:.4f}, p = {p:.4f}")
        return True

def are_normal(d1, d2, alpha):
    """
    Method for performing stats.normaltest on two input samples and returns boolian if sample distibutions are normal 
    As return we get a boolian if sample distibutions are normal.

    Args:
        d1, d2 (pd.DataSeries): input sample data
        alpha (float): alpha value as proportion 

    Returns:
        boolian: true if sample distributions are normal (p > alpha), false otherwise (p <= alpha)
    """
    ### check if sample distributions are normal   
    for d in [d1, d2]:
        k2, p = stats.normaltest(d)
        if p <= alpha:
            print(f"sample distribution not normal")
            return False
    return True

def t_test(d1, d2, alpha):
    """
    Method for performing t-test on two sample distributions:
        Welch's t-test for unequal variance
        independent t-test for equal variance 
    For non-normal distributions of groups, sample size has to be of large size (appr. > 50) 
    to get a valid t-test on a non normal distribution.
        
    Args:
        d1, d2 (pd.DataSeries): input sample data
        alpha (float): alpha value as proportion 
    
    Returns:
        statistic (float): test statistic
        p (float): p value for marginal significance
        interpretation (string): interpretation of statistical significance for the two means
    """
    ### check of equal variance
    equal_var = is_equal_variance(d1, d2, alpha)
    
    ### check if group values are normally distributed
    normal = are_normal(d1, d2, alpha)    
    
    ### select appropriate test
    if not equal_var:
        print("Welch's t-test on non-normal distributed samples with unequal variances:")
        statistic, p = stats.ttest_ind(d1, d2, equal_var = equal_var)
    else:
        print("Independent t-test:")
        statistic, p = stats.ttest_ind(d1, d2, equal_var = equal_var) 
        
    ### t-test interpretation 
    significant = "statistically significant" if p <= alpha else "not statistically significant"
    interpretation = f"mean difference between groups is {significant}."
    
    return statistic, p, interpretation 

def compare_group_means(d1, d2, alpha):
    """
    Method for performing sample means comparison with statistics summary of samples, 
    mean difference calulation, effect size test and t-test. 
    
    Args:
        d1, d2 (pd.DataSeries): input sample data
        alpha (float): alpha value as proportion 
    
    Returns: 
        print statements for statistics of samples, mean difference calulation, effect size test and t-test
    """
    
    ### aggregate statistics
    print ("sample statistics:")
    d1_describe = d1.agg(["count", "mean", "median", "std"])
    d2_describe = d2.agg(["count", "mean", "median", "std"])
    display(d1_describe, d2_describe)
    
    ### calculate difference between group means
    diff = d1.mean() - d2.mean()
    diffp = 100.0 * abs(diff) / d1.mean()
    print(f"mean difference between groups: {diff:.4f}({diffp:.2f}%)\n")
    
    ### Perform t-test
    statistic, p, interpretation = t_test(d1, d2, alpha)
    print(f"statistic = {statistic:.4f} | p-value = {p:.4f} =>> {interpretation}")

# def create_group(df, rf, value):
#     """
#     for group creation, namely success and failure
    
#     Args:
#         df (pd.DataSeries): the data dataframe to get the view and complete 
#         rf (pd.DataSeries): the ref dataframe to get the value 
#         value (column name of panda series): of the reference
    
#     Returns: 
#         two groups, success and failure dataframe
#     """    
#     list_s = []
#     list_f = []
    
#     for index, item in df:
        
    
#     return success, fail

Comparing the 4 main client side factors namely (age, income, gender, length of membership)

## Offer

###### Which is the most effective channel in dispensing out information?

First let us have an overview of the dataframe

In [ ]:
plt.subplots(figsize=(20, 12))

value = []
for i in range(0,portfolio.shape[0]):
    value.append(1.0)
    value.append(portfolio_df.Viewed.iloc[i] /portfolio_df.Offered.iloc[i])
    value.append(portfolio_df.Complete.iloc[i] /portfolio_df.Offered.iloc[i])

color = [] 
for i in range(0,portfolio.shape[0]):
    color.append("tab:olive")
    color.append("tab:orange")
    color.append("tab:cyan")
    
index = []
for i in range(0,portfolio.shape[0]):
    index.append("campaign_" + str(i) + "_offered")
    index.append("campaign_" + str(i) + "_viewed")
    index.append("campaign_" + str(i) + "_completed")

plt.bar(index, height=value, color = color)

plt.xticks(rotation=45)

plt.title("Distribution of Customer Age Among The Income Group\n",
         fontweight ="bold")

plt.ylabel("No of People")
plt.xlabel("Customer Age group")
plt.show()

success  = pd.DataFrame()
success['offer_type'] = portfolio_df.offer_type
success['difficulty'] = portfolio_df.difficulty
success['reward']     = portfolio_df.reward
success['duration']   = portfolio_df.duration
success['complete']   = value[2::3]

print(success)


Campaign 6,7,9, namely offer types: {discount, discount, bogo}, have above 60% completion rate. 

Observation
<ul>
    <li> At first glance, it seem that difficulty matters most in almost all types of offer. The more difficult it is, the lower is the completion rate,no matter the reward and time duration of the reward
    <li> The type of offer seems to rank the next in the terms of effect, with discount offer being higher
    <li> The influence of rewards seem to matter after next
    <li> Duration seem to matter the least in all of this effect
</ul>

But is that so?

Let us go back to the question in which media channel offers the greatest attention. We would say media has the effect of bring attention to, ie in this case for customer to view the offer. Whether to complete it, would depend on the reward, difficulty of the offer and nature of customer

In [ ]:
plt.subplots(figsize=(8, 4))

influence_web    = (portfolio_df.Viewed * portfolio_df.web).sum() / (portfolio_df.Offered * portfolio_df.web).sum()  * 100
influence_mobile = (portfolio_df.Viewed * portfolio_df.mobile).sum() / (portfolio_df.Offered * portfolio_df.mobile).sum() * 100
influence_social = (portfolio_df.Viewed * portfolio_df.social).sum() / (portfolio_df.Offered * portfolio_df.social).sum() * 100

influence = [influence_web, influence_mobile, influence_social] 
label = ['influence_web', 'influence_mobile', 'influence_social']

color = ['tab:cyan', 'tab:orange', 'tab:olive']

plt.xticks(rotation=45)
plt.title('Plot of Viewing Rate (%) of Various Media Channel')

plt.bar(label, influence, color=color)

### Lastly to show
plt.show()

Not surprising, customer will respond more to their social media compared to other form of communication. This maybe because of social media give a more human touch and directed advertisement effect compared to mobile. Mobile, though personal, do not give one the personal touch

In [ ]:
profile_df_m1 = profile_df.loc[ (profile_df.gender =='M') & (profile_df.age <= 45) ]
profile_df_f1 = profile_df.loc[ (profile_df.gender =='F') & (profile_df.age <= 45) ]
profile_df_m2 = profile_df.loc[(profile_df.gender =='M') & ( (profile_df.age > 45) & (profile_df.age <= 58))  ]
profile_df_f2 = profile_df.loc[ (profile_df.gender =='F') & ( (profile_df.age > 45) & (profile_df.age <= 58))  ]
profile_df_m3 = profile_df.loc[(profile_df.gender =='M') & ( (profile_df.age > 58) & (profile_df.age <= 73))]
profile_df_f3 = profile_df.loc[ (profile_df.gender =='F') & ( (profile_df.age > 58) & (profile_df.age <= 73)) ]
profile_df_m4 = profile_df.loc[(profile_df.gender =='M') & (profile_df.age > 73)]
profile_df_f4 = profile_df.loc[ (profile_df.gender =='F') & (profile_df.age > 73) ]

ratio_m1 = profile_df_m1.Viewed.sum() / profile_df_m1.Offered.sum()  * 100
ratio_f1 = profile_df_f1.Viewed.sum() / profile_df_f1.Offered.sum()  * 100
ratio_m2 = profile_df_m2.Viewed.sum() / profile_df_m2.Offered.sum()  * 100
ratio_f2 = profile_df_f2.Viewed.sum() / profile_df_f2.Offered.sum()  * 100
ratio_m3 = profile_df_m3.Viewed.sum() / profile_df_m3.Offered.sum()  * 100
ratio_f3 = profile_df_f3.Viewed.sum() / profile_df_f3.Offered.sum()  * 100
ratio_m4 = profile_df_m4.Viewed.sum() / profile_df_m4.Offered.sum()  * 100
ratio_f4 = profile_df_f4.Viewed.sum() / profile_df_f4.Offered.sum()  * 100

influence = [ratio_m1, ratio_f1, 
             ratio_m2, ratio_f2,
             ratio_m3, ratio_f3, 
             ratio_m4, ratio_f4] 
label    =   ['ratio_m1', 'ratio_f1', 
             'ratio_m2', 'ratio_f2',
             'ratio_m3', 'ratio_f3', 
             'ratio_m4', 'ratio_f4'] 

color = ['tab:cyan', 'tab:orange',
        'tab:cyan', 'tab:orange',
        'tab:cyan', 'tab:orange',
        'tab:cyan', 'tab:orange']

plt.xticks(rotation=45)
plt.title('\n\n Plot of Viewing Rate (%) of Various Media Channel ')

plt.bar(label, influence, color=color)

The number of offers for each campaign is largely the same. So we can say, it is safe to compare between each 

It seem to show that social media has great influence on the client. Most of the client actually responded to the campaign when it is hosted on social media

###### Which group (age, income, membership length) of customers appear to be more responsive towards the promotion

The customer response for Young Male is particularly poor. While the rest of the group exhibits at least 38% of offer uptake rate, Young Male group seem to have relatively 'poorer' uptake rate at about 28.2%

###### Is there link between the offer given and the target group?

###### Which type of promotional offer most appeal to which group of customer?

## Modeling and Evaluation

###### Can we build a recommendation engine to recommend promotional offer with good uptake rate to new customers based on their demographical data age, income, registration date and gender? 

Selecting the 

Selection of Features

There is a plethora of features within the dataset we would need to fish out the most relevant features in our studies

In [ ]:
plt.figure(figsize=(9, 6))
### Plot Heatmap with feature multicollinearity
sns.heatmap(master_offer2.corr(), annot = True, vmin = -1, vmax = 1, fmt= ".2f", cmap = "coolwarm")
plt.title("Feature Multicollinearity - master_offer1", fontsize = "x-large")
plt.xticks(rotation = 45, ha = "right");

In [ ]:
def classifier(classifier, offer_id, X_train, X_test, y_train, y_test):
    """
    Method for training classifier and predicting on test set. 
    Output are: the trained classifier and the evaluation metrics accuracy and f1-score

    Args:
        classifier: classifier ith parameters, if procurable including GridSearchCV for parameter tuning
        offer_id (int): integer with offer_id according master dataframe
        X_train (pd.DataFrame): DataFrame with feature columns of train set
        y_train (pd.DataSeries): Series with target column of train set
        X_test (pd.DataFrame): DataFrame with feature columns of test set
        y_test (pd.DataSeries): Series with target column of test set

    Returns:
        clf: trainied classifier
        accuracy (float): accuracy score 
        f1 (float): f1-score
        recall (float): recall score
        precision (float): precision score
    """
    ### train classifier
    clf.fit(X_train, y_train)
    
    ### predict on test set
    pred = clf.predict(X_test)
    ### accuracy score
    accuracy = round(accuracy_score(y_test, pred)*100,2)
    ### f1 score
    f1 = round(f1_score(y_test, pred)*100,2)
    ### recall score
    recall = round(recall_score(y_test, pred)*100,2)
    ### precision score
    precision = round(precision_score(y_test, pred)*100,2)
    
    print("#######################################################")
    ### confusion matrix
    cm = confusion_matrix(y_test, pred)
    print("Offer {} - confusion matrix:".format(offer_id))
    print(cm, "\n")
    
    ### classification report
    cr = classification_report(y_test, pred, target_names = ["0", "1"])
    print("Offer {} - classification report:".format(offer_id))
    print(cr)  
    
    print ("Offer {}:".format(offer_id),
           "Accuracy: {} % | F1-score: {} % \n\
        Recall: {} % | Precision: {} %".format(accuracy, f1, recall, precision),"\n")
    
    return clf, accuracy, f1, recall, precision

In [ ]:
%%time

### define parameters and classifier
parameters = {"bootstrap": [True],
              "max_depth": [2, 6, 10], 
              "max_features": [0.5, 1, 2], 
              "min_samples_leaf": [1, 5], 
              "min_samples_split": [2, 5], 
              "n_estimators": [10, 20]}
RF = RandomForestClassifier()
clf = GridSearchCV(RF, parameters, scoring = "roc_auc", cv = 4, n_jobs = 4, verbose = 2)

### train classifier, predict on test set and compute evaluation metrics
rf_1, acc_rf_1, f1_rf_1, r_rf_1, p_rf_1 = classifier(clf, 1, X1_train, X1_test, y1_train, y1_test)
rf_2, acc_rf_2, f1_rf_2, r_rf_2, p_rf_2 = classifier(clf, 2, X2_train, X2_test, y2_train, y2_test)
rf_4, acc_rf_4, f1_rf_4, r_rf_4, p_rf_4 = classifier(clf, 4, X4_train, X4_test, y4_train, y4_test)
rf_5, acc_rf_5, f1_rf_5, r_rf_5, p_rf_5 = classifier(clf, 5, X5_train, X5_test, y5_train, y5_test)
rf_6, acc_rf_6, f1_rf_6, r_rf_6, p_rf_6 = classifier(clf, 6, X6_train, X6_test, y6_train, y6_test)
rf_7, acc_rf_7, f1_rf_7, r_rf_7, p_rf_7 = classifier(clf, 7, X7_train, X7_test, y7_train, y7_test)
rf_9, acc_rf_9, f1_rf_9, r_rf_9, p_rf_9 = classifier(clf, 9, X9_train, X9_test, y9_train, y9_test)
rf_10, acc_rf_10, f1_rf_10, r_rf_10, p_rf_10 = classifier(clf, 10, X10_train, X10_test, y10_train, y10_test)

## Deployment

In [ ]:
### add dummy columns for gender type categories. 
dummies_gender = pd.get_dummies(customer_data["gender"], columns = ["gender"])
customer_data = pd.concat([customer_data, dummies_gender.set_index(customer_data.index)], axis = 1)
del customer_data["gender"]

### convert Dtype from object to datetime64
customer_data["became_member_on"] = pd.to_datetime(customer_data["became_member_on"])
### convert Dtype from datetime64 to ordinal
customer_data["became_member_on"] = customer_data["became_member_on"].map(datetime.datetime.toordinal)

### scale features of new customer data
def scale_feature(ref_col, scale_col):
    """
    Method for scaling one selected features column on min max values of a reference dataframe (i.e. master)

    Args:
        ref_df (pd.Series): Series with values whereof min and max values are taken for scaling
        scale_df (pd.Series): Series with values of the same category as ref_df to be scaled

    Returns:
        scaled_values (pd.Series): Series with scaled values
    """
    scaled_values = (scale_col - ref_col.min()) / (ref_col.max() - ref_col.min())

    return scaled_values

### scale columns age, become_member_on, income_year of new customer data
customer_data["age"] = scale_feature(master["age"], customer_data["age"])
customer_data["became_member_on"] = scale_feature(master["became_member_on"], customer_data["became_member_on"])
customer_data["income year (USD)"] = scale_feature(master["income year (USD)"], customer_data["income year (USD)"])


In [ ]:
### create dataframe with new customer data for prediction promomtion success
customer_id = ["customer_01", "customer_02", "customer_03", "customer_04", "customer_05", 
               "customer_06", "customer_07", "customer_08", "customer_09", "customer_10"]
age = [20, 25, 30, 35, 45, 50, 55, 60, 70, 80 ] # range: 18 - 101
became_member_on = ["2013-07-29", "2016-07-29", "2018-07-26", "2013-07-29", "2016-07-29",
                   "2018-07-26", "2013-07-29", "2015-07-29", "2017-07-29", "2017-07-29"] # range: 2013-07-29 - 2018-07-26
income_year = [60000, 40000, 100000, 100000, 70000, 40000, 30000, 80000, 40000, 100000] # range: 30000 - 120000
gender = ["M", "F", "F", "F", "O", "M", "F", "F", "F", "M"]

customer_data = pd.DataFrame(
    {"age": age,
     "became_member_on": became_member_on,
     "income year (USD)": income_year,
     "gender": gender,
     }, index = customer_id)
display(customer_data)

In [ ]:


### predict promotion success for new customer data with selected trained classifier
pred_offer_1 = rf_1.predict(customer_data)
pred_offer_2 = knn_2.predict(customer_data)
pred_offer_4 = rf_4.predict(customer_data)
pred_offer_5 = dt_5.predict(customer_data)
pred_offer_6 = svm_6.predict(customer_data)
pred_offer_7 = gnb_7.predict(customer_data)
pred_offer_9 = knn_9.predict(customer_data)
pred_offer_10 = rf_10.predict(customer_data)

### create dataframe
pred_offer = pd.DataFrame(
    {"offer_01": pred_offer_1,
     "offer_02": pred_offer_2,
     "offer_04": pred_offer_4,
     "offer_05": pred_offer_5,
     "offer_06": pred_offer_6,
     "offer_07": pred_offer_7,
     "offer_09": pred_offer_9,
     "offer_10": pred_offer_10,
     }, index = customer_id)

pred_offer["sum success"] = pred_offer[["offer_01", "offer_02", "offer_04", 
                                        "offer_05", "offer_06", "offer_07", 
                                        "offer_09", "offer_10"]].sum(axis=1)

display("Overview table with promotion success prediction for test customers", pred_offer)




# References 

<ol>
    <li>Starbuck Wikipaedia, https://en.wikipedia.org/wiki/Starbucks</li>
    <li>Starbuck Company Website, https://www.starbucks.com/about-us/ </li>
    <li>https://digital.hbs.edu/platform-digit/submission/starbucks-mobile-app-a-winner-in-bridging-the-retail-digital-divide/ </li>
    <li>List of the verified oldest people, <https://en.wikipedia.org/wiki/List_of_the_verified_oldest_people</li>
    </ol>